In [ ]:
# --------------------------------------
# 1. Import Libraries
# --------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from imblearn.over_sampling import SMOTE

import gradio as gr
import os

In [ ]:
# --------------------------------------
# 2. Load and Inspect Dataset
# --------------------------------------
def load_data(path: str, target_column: str = "Class") -> pd.DataFrame:
    df = pd.read_csv(path)
    if target_column not in df.columns:
        # Raise a more informative KeyError if the target column is not found
        raise KeyError(f"Target column '{target_column}' not found. Available columns: {df.columns.tolist()}")
    print("Original Class Distribution:")
    print(df[target_column].value_counts())
    return df

In [ ]:
# 3. Preprocessing
# --------------------------------------
def preprocess_data(df: pd.DataFrame, target_column: str = "Is_Fraud"): # Updated default target_column
    if target_column not in df.columns:
        raise KeyError(f"Target column '{target_column}' not found in DataFrame.")

    # Drop non-numeric columns that are not features or the target
    # Assuming 'Transaction_ID' and 'Customer_ID' are columns to drop
    cols_to_drop = ['Transaction_ID', 'Customer_ID']
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # Identify and drop non-numeric columns from X before scaling and training
    non_numeric_cols = X.select_dtypes(exclude=np.number).columns.tolist()
    if non_numeric_cols:
        print(f"Dropping non-numeric columns: {non_numeric_cols}")
        X = X.drop(columns=non_numeric_cols)

    cols_to_scale = [col for col in ['Time', 'Amount'] if col in X.columns]
    scaler = StandardScaler() if cols_to_scale else None
    if scaler:
        # Ensure columns to scale are numeric before applying scaler
        numeric_cols_to_scale = X[cols_to_scale].select_dtypes(include=np.number).columns.tolist()
        if numeric_cols_to_scale:
             X[numeric_cols_to_scale] = scaler.fit_transform(X[numeric_cols_to_scale])
        else:
             print("Warning: 'Time' or 'Amount' columns are not numeric and cannot be scaled.")


    smote = SMOTE(random_state=42)
    try:
        # Ensure X and y are suitable for SMOTE (all numeric for X, numerical or categorical for y)
        if X.select_dtypes(exclude=np.number).empty:
            X_res, y_res = smote.fit_resample(X, y)
            print("Balanced Class Distribution:")
            print(pd.Series(y_res).value_counts())
        else:
            print("SMOTE cannot be applied due to non-numeric features in X after dropping. Proceeding without resampling.")
            X_res, y_res = X, y

    except ValueError as e:
        print(f"SMOTE error: {e}. Proceeding without resampling.")
        X_res, y_res = X, y
    return X_res, y_res, scaler


In [ ]:
# 4. Train Model
# --------------------------------------
def train_model(X, y):
    if X.empty or y.empty or len(X) != len(y):
        raise ValueError("Training data is empty or mismatched.")

    # Ensure X contains only numeric data before splitting and training
    if not X.select_dtypes(exclude=np.number).empty:
        non_numeric_cols_in_X = X.select_dtypes(exclude=np.number).columns.tolist()
        raise ValueError(f"Input data for training contains non-numeric columns after preprocessing: {non_numeric_cols_in_X}. Please ensure all feature columns are numeric.")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model, X_test, y_test

# Rest of the code remains the same...


In [ ]:
# 5. Evaluation
# --------------------------------------
def evaluate_model(model, X_test, y_test):
    if X_test.empty or y_test.empty:
        print("Warning: Empty test data.")
        return
    y_pred = model.predict(X_test)
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    if hasattr(model, 'predict_proba') and len(np.unique(y_test)) > 1:
        # Check if the target variable has more than one class before calculating ROC-AUC
        if len(np.unique(y_test)) > 1:
             # Ensure X_test is numeric before predicting probabilities
             if X_test.select_dtypes(exclude=np.number).empty:
                 print("ROC-AUC Score:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
             else:
                 print("ROC-AUC Score cannot be calculated: X_test contains non-numeric columns.")
        else:
             print("ROC-AUC Score cannot be calculated as there is only one class in y_test.")
    else:
        print("ROC-AUC Score cannot be calculated (model does not have predict_proba or only one class).")



In [ ]:
# 6. Gradio Interface
# --------------------------------------
def create_interface(model, scaler):
    if model is None:
        print("Cannot create Gradio interface: Model is not trained.")
        return

    def predict_fraud(time, amount, *features):
        try:
            scaled_vals = scaler.transform([[time, amount]])[0] if scaler else [time, amount]
        except Exception as e:
            print(f"Scaling error: {e}")
            scaled_vals = [time, amount]

        # Ensure input data has the same number of features as the training data
        if hasattr(model, 'feature_names_in_'):
            all_model_features = model.feature_names_in_.tolist()
            # Reconstruct the input list based on model's feature names
            # Handle Time and Amount first, then add the V features in the order expected by the model
            input_data_dict = {'Time': scaled_vals[0], 'Amount': scaled_vals[1]}
            # Assuming 'features' correspond to V1, V2, ... V28 in order
            v_feature_names_in_model = [f for f in all_model_features if f.startswith('V')]
            for i, v_feat_name in enumerate(v_feature_names_in_model):
                if i < len(features):
                    input_data_dict[v_feat_name] = features[i]
                else:
                    # Handle case where fewer V features are provided than expected
                    # This might indicate an issue with the interface inputs or the model's expectations
                    print(f"Warning: Missing value for expected feature '{v_feat_name}'. Using 0 as placeholder.")
                    input_data_dict[v_feat_name] = 0 # Placeholder, consider better handling

            # Create the input array ensuring the order matches the model's feature_names_in_
            input_data_list = [input_data_dict.get(feat, 0) for feat in all_model_features] # Use 0 as placeholder for missing features
            input_data = np.array([input_data_list])

            if input_data.shape[1] != len(all_model_features):
                 raise ValueError(f"Input shape mismatch. Expected {len(all_model_features)} features, got {input_data.shape[1]}. Ensure all required features are provided and in the correct format.")

        else:
            # Fallback if feature_names_in_ is not available (e.g., older sklearn versions)
            # Assuming the order is Time, Amount, then V1 to V28
            input_data = np.array([[scaled_vals[0], scaled_vals[1]] + list(features)])
            # Add a check here if possible based on the expected number of features

        # Ensure input_data is numeric before prediction
        try:
             input_data = input_data.astype(float)
        except ValueError as e:
             return "Error", f"Input data contains non-numeric values: {e}"


        prediction = model.predict(input_data)[0]
        # Check if the model has predict_proba and the target has two classes
        if hasattr(model, 'predict_proba') and model.classes_.shape[0] > 1:
             prob = model.predict_proba(input_data)[0][1]
             confidence_text = f"Confidence: {prob:.2%}"
        else:
             prob = None
             confidence_text = "Confidence information not available (model does not have predict_proba or target is not binary)."


        return "Fraudulent" if prediction == 1 else "Legitimate", confidence_text

    if hasattr(model, 'feature_names_in_'):
        all_features_in_model = model.feature_names_in_.tolist()
        # Assuming Time and Amount are the first two features if feature_names_in_ is available
        # Also exclude 'Transaction_ID' and 'Customer_ID' which were dropped during preprocessing
        # and are not expected as inputs to the Gradio interface
        v_features_in_model = [f for f in all_features_in_model if f.startswith('V')]
        # Ensure the order for Gradio inputs matches the model's expectations (Time, Amount, then V features in order)
        gr_input_feature_names = [f for f in ['Time', 'Amount'] if f in all_features_in_model] + v_features_in_model
    else:
        # Fallback for older sklearn versions, assuming Time, Amount, then V1 to V28
        gr_input_feature_names = ['Time', 'Amount'] + [f"V{i}" for i in range(1, 29)] # Assuming V1 to V28
        # This fallback might be inaccurate if the actual data has other non-V columns besides Time and Amount

    gr_inputs = [gr.Number(label=f) for f in gr_input_feature_names]
    gr_outputs = [gr.Text(label="Prediction"), gr.Text(label="Confidence")]


    gr.Interface(
        fn=predict_fraud,
        inputs=gr_inputs,
        outputs=gr_outputs,
        title="💳 AI-Powered Credit Card Fraud Detection",
        description="Input transaction details to check if it's fraudulent or legitimate."
    ).launch()




In [ ]:
# 7. Run Full Pipeline
# --------------------------------------
if __name__ == "__main__":
    csv_file_path = "credit_card_fraud_dataset.csv"
    if not os.path.exists(csv_file_path):
        print(f"Error: Dataset file not found at '{csv_file_path}'.")
    else:
        # Change target_column from "Class" to "Is_Fraud"
        df = load_data(csv_file_path, target_column="Is_Fraud")
        # Change target_column from "Class" to "Is_Fraud"
        X_res, y_res, scaler = preprocess_data(df, target_column="Is_Fraud")
        model, X_test, y_test = train_model(X_res, y_res)
        evaluate_model(model, X_test, y_test)
        create_interface(model, scaler)

Original Class Distribution:
Is_Fraud
0    3793
1     207
Name: count, dtype: int64
Dropping non-numeric columns: ['Timestamp', 'Merchant_ID', 'Merchant_Category', 'Transaction_Type', 'Location_City', 'Location_Country', 'Device_ID', 'Channel', 'Is_3DS_Authenticated', 'Previous_Fraud_Flag']
Balanced Class Distribution:
Is_Fraud
0    3793
1    3793
Name: count, dtype: int64

Confusion Matrix:
[[445 330]
 [280 463]]

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.57      0.59       775
           1       0.58      0.62      0.60       743

    accuracy                           0.60      1518
   macro avg       0.60      0.60      0.60      1518
weighted avg       0.60      0.60      0.60      1518

ROC-AUC Score: 0.690024747101984


/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1027: UserWarning: Expected at least 2 arguments for function <function create_interface.<locals>.predict_fraud at 0x7922119f09a0>, received 1.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://69354c9b2702f64e89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
